In [61]:
#import various libraries which we will use
import pandas as pd
import numpy as np

#sk learn moduels 
from sklearn import cross_validation, ensemble

In [62]:
# Read the training / testing dataset
titanic_train = pd.read_csv("data/train.csv")
titanic_test = pd.read_csv("data/test.csv")

#Inputs into model
X = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
#outcome
y = ["Survived"]

In [63]:
titanic_test[X].head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [64]:
def encode_data(titanic):
    #set the gender
    titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
    titanic.loc[titanic["Sex"] == "female", "Sex"] = 1
    
    #fill in missing values
    titanic["Embarked"] = titanic["Embarked"].fillna("S")

    #encode embarked
    titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
    titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
    titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2
    
    titanic['Age'] = titanic["Age"].fillna(titanic.Age.median())
    titanic['Fare'] = titanic["Fare"].fillna(titanic.Fare.median())
    
    return titanic

In [65]:
titanic_train = encode_data(titanic_train)
titanic_test = encode_data(titanic_test)

In [66]:
titanic_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,NaN,0


In [73]:
rfc = ensemble.RandomForestClassifier(random_state=1, n_estimators=100, min_samples_split = 6, min_samples_leaf = 2)

scores_rfc = cross_validation.cross_val_score(
    rfc,
    titanic_train[X],
    titanic_train["Survived"],
    cv=3
)

print(scores_rfc.mean())

0.820426487093


In [71]:
gbc = ensemble.GradientBoostingClassifier(random_state=5, n_estimators=200, min_samples_split = 6, min_samples_leaf = 3)

scores_gbc = cross_validation.cross_val_score(
    gbc,
    titanic_train[X],
    titanic_train["Survived"],
    cv=3
)

print(scores_gbc.mean())

0.811447811448


In [75]:
rfc.fit(titanic_train[X],titanic_train["Survived"])
predictions = rfc.predict(titanic_test[X])

def output_file(predictions, file_name):
    submission = pd.DataFrame({
            "PassengerId": titanic_test["PassengerId"],
            "Survived": predictions
        })
    
    submission.to_csv(file_name, index=False)